# Problem 198: Ambiguous Numbers

A best approximation to a real number $x$ for the denominator bound $d$ is a rational number $\frac r s$ (in reduced form) with $s \le d$, so that any rational number $\frac p q$ which is closer to $x$ than $\frac r s$ has $q \gt d$.

Usually the best approximation to a real number is uniquely determined for all denominator bounds. However, there are some exceptions, e.g. $\frac 9 {40}$ has the two best approximations $\frac 1 4$ and $\frac 1 5$ for the denominator bound $6$.
We shall call a real number $x$ <dfn>ambiguous</dfn>, if there is at least one denominator bound for which $x$ possesses two best approximations. Clearly, an ambiguous number is necessarily rational.

How many ambiguous numbers $x=\frac p q, 0 \lt x \lt \frac 1 {100}$, are there whose denominator $q$ does not exceed $10^8$?

In [36]:
from dataclasses import dataclass


@dataclass
class Fraction:
    num: int
    denum: int

    @property
    def as_float(self) -> float:
        if self.denum == 0:
            return float("inf")
        else:
            return self.num / self.denum

    @property
    def value(self) -> tuple[int, int]:
        return (self.num, self.denum)

    def __repr__(self) -> str:
        return f"{self.num}/{self.denum}"


zero = Fraction(0, 1)
infty = Fraction(1, 0)
print(zero.as_float)
print(zero)
print(infty)


@dataclass
class SternBrocotBranch:
    left: "Fraction"
    right: "Fraction"

    def __post_init__(self):
        self.value = Fraction(
            self.left.num + self.right.num, self.left.denum + self.right.denum
        )

    @property
    def left_child(self) -> "SternBrocotBranch":
        return SternBrocotBranch(self.left, self.value)

    @property
    def right_child(self) -> "SternBrocotBranch":
        return SternBrocotBranch(self.value, self.right)

    @property
    def children(self) -> tuple["SternBrocotBranch", "SternBrocotBranch"]:
        return (self.left_child, self.right_child)

    def get_ambigous_fraction(self, lft: "Fraction", rgt: "Fraction") -> "Fraction":
        num = lft.num * rgt.denum + rgt.num * lft.denum
        denum = 2 * lft.denum * rgt.denum
        return Fraction(num, denum)

    @property
    def left_ambigous(self) -> "Fraction":
        return self.get_ambigous_fraction(self.left, self.value)

    @property
    def right_ambigous(self) -> "Fraction":
        return self.get_ambigous_fraction(self.value, self.right)

    def __repr__(self) -> str:
        return f"SternBrocotLeaf({self.value})"


root = SternBrocotBranch(zero, infty)
print(root)
print(root.children)
print([r.children for r in root.children])

0.0
0/1
1/0
SternBrocotLeaf(1/1)
(SternBrocotLeaf(1/2), SternBrocotLeaf(2/1))
[(SternBrocotLeaf(1/3), SternBrocotLeaf(2/3)), (SternBrocotLeaf(3/2), SternBrocotLeaf(3/1))]


In [48]:
xmax = Fraction(1, 100)
denom_bound = 10**8 + 1

zero = Fraction(0, 1)
infty = Fraction(1, 0)

leafs = [SternBrocotBranch(zero, infty)]
counter = 0
# ambigous = []
while len(leafs) > 0:
    _leafs = []
    for leaf in leafs:
        # ambigous numbers
        _min_denom = denom_bound
        for amb in [leaf.left_ambigous, leaf.right_ambigous]:
            _min_denom = min(amb.denum, _min_denom)
            if amb.denum < denom_bound and amb.as_float < xmax.as_float:
                counter += 1
                # ambigous.append(amb)

        # add next row
        (left_child, right_child) = leaf.children
        if _min_denom < denom_bound:
            _leafs.append(left_child)
        if right_child.value.as_float < xmax.as_float and _min_denom < denom_bound:
            _leafs.append(right_child)
    leafs = _leafs
print(counter)

52374425
